<a href="https://colab.research.google.com/github/cruz-marco/pyspark_course/blob/main/pyspark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação e configuração Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/content/spark-3.2.3-bin-hadoop3.2'

!pip install -q findspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

# DataFrames

- Tabelas com linhas e colunas;
- Imutáveis;
- Schema conhecido;
- Linhagem Preservada;
- Colunas podem ter tipos diferentes;
- Podemos agrupar, ordenar e filtrar;
- Spark otimiza análises usando planos de execução (DAG's)

## Lazy Evaluation
> O processamento da transformação só ocorre quando há uma ação: 

## Ações:
> (reduce, collect, count, first, take, takeSample, takeOrdered, saveAsTestFile, saveAsSequenceFile, saveAsObjectFile, countByKey, foreach)

## Transformações:
> (map, filter, flatMap, mapPartitions, mapPartitionsWithIndex, sample, union, intersection, distinct, groupByKey, reduceByKey, aggregateByKey, sortByKey, join, cogroup, cartesian, pipe, coalesce, repartition, repartitionAndSortWithinPartitions)


### Criando um [DataFrame](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html#pyspark.sql.DataFrame) de exemplo:
- [spark.createDataFrame()](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.createDataFrame.html)

In [2]:
data = [("Pedro", 10), ("Maria", 20), ("José", 40)] #Dados a serem inseridos na tabela
df1 = spark.createDataFrame(data) #instanciando o DataFrame
df1.show() #Comando para mostar o frame, pode recer um parâmetro com o número.

+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| José| 40|
+-----+---+



In [3]:
schema = "Id INT, Nome STRING" #definindo um schema a ser usado no DataFrame
data_2 = [(1, "Pedro"), (2, "Maria")]

df2 = spark.createDataFrame(data_2, schema=schema)

df2.show()

+---+-----+
| Id| Nome|
+---+-----+
|  1|Pedro|
|  2|Maria|
+---+-----+



### Pacote de [funções](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html):

In [24]:
from pyspark.sql import functions as f #Importando a biblioteca de funções
schema_2 = "produtos STRING, vendas INT"
vendas = [("Caneta", 10), ("Lápis", 20), ("Caneta", 40)]

df3 = spark.createDataFrame(vendas, schema_2)

df3.show()

+--------+------+
|produtos|vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [25]:
"""
Diferente do Pandas, as funções ficam em um pacote a parte, 
logo, temos que fazer o import ou do pacote, ou da função 
específica. Por exemplo, a sum() utilizada com o método de 
agregação, conforme mostrado abaixo.
"""

agrupado = df3.groupBy("produtos")\
            .agg(f.sum("vendas"))

agrupado.show()

+--------+-----------+
|produtos|sum(vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [26]:
df3.select("produtos").show() # Selecionando uma única coluna no DataFrame, por exemplo.

+--------+
|produtos|
+--------+
|  Caneta|
|   Lápis|
|  Caneta|
+--------+



In [27]:
df3.select("produtos", "vendas", f.expr("vendas * 0.2")).show() 
"""
A Função EXPR cria uma expressão que pode ser usada para criar mais uma linha
no dataframe, enriquecendo a nossa análise. Ela recebe uma string com a
expressão a ser processada.
"""

+--------+------+--------------+
|produtos|vendas|(vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



'\nA Função EXPR cria uma expressão que pode ser usada para criar mais uma linha\nno dataframe, enriquecendo a nossa análise. Ela recebe uma string com a\nexpressão a ser processada.\n'

In [28]:
display(
    df3.schema, # ver a estrutura das colunas
    df3.columns # ver os nomes das colunas
)

StructType(List(StructField(produtos,StringType,true),StructField(vendas,IntegerType,true)))

['produtos', 'vendas']

### Carregando dados de duas maneiras distintas no Spark (DataFrames)

Usando o spark.read.csv:

In [29]:
from pyspark.sql.types import *

arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"
# esquema de colunas, colocando seu nome e tipo, em sequência, para ser usado
# como referência na leitura dos dados em CSV, para podermos escolher
# os tipos de dados e o nome das colunas, visto que o CSV em questão não possui
# cabeçalho.

despachantes = spark.read.csv("/content/drive/MyDrive/Datasets/pyspark_course/despachantes.csv", 
                              header=False, schema=arqschema)


In [30]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [31]:
despachantes.groupBy("cidade")\
.agg(f.sum("vendas")).show()
# Exemplo de groupBy

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|  Santa Maria|         68|
|Novo Hamburgo|         34|
| Porto Alegre|        223|
+-------------+-----------+



Usando o spark.read.load:

In [32]:
desp_autoschema = spark.read.load("/content/drive/MyDrive/Datasets/pyspark_course/despachantes.csv",
                                  header=False, format="csv", sep=",",
                                  inferSchema=True)
# forma um tanto mais sucinta, é necessário informar o formato e permitir o 
# parâmetro inferSchema (valor True) para que o próprio spark deduza o tipo
# de dados sozinho. É uma boa prática informar qual o tipo de separador SEP,
# pois no Brasil, usamos a vírgula para declarar a parte decimal de um número
# não inteiro.

In [33]:
desp_autoschema.show()
# como o arquivo não tem cabeçalho, o spark nomeia as colunas automaticamente.

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [34]:
display(
    despachantes.schema, # comparação dos schemas declarados e inferidos.
    "------------------",
    desp_autoschema.schema
)


StructType(List(StructField(id,IntegerType,true),StructField(nome,StringType,true),StructField(status,StringType,true),StructField(cidade,StringType,true),StructField(vendas,IntegerType,true),StructField(data,StringType,true)))

'------------------'

StructType(List(StructField(_c0,IntegerType,true),StructField(_c1,StringType,true),StructField(_c2,StringType,true),StructField(_c3,StringType,true),StructField(_c4,IntegerType,true),StructField(_c5,StringType,true)))

### Fazendo consultas

> Select, Where, OrderBy, Distinct são cláusulas SQL que no PySpark são métodos do objeto DataFrame, diferentemente do pandas, temos que usar o SELECT para selecionar as colunas que nos interessam, no lugar de exibirmos todas.

In [36]:
despachantes.select("id", "nome", "vendas")\
        .where(f.col("vendas") > 20)\
        .show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+



> A cláusula WHERE deve ser usada juntamente com a função COL do pacote de funções SQL do PySpark. As condições podem ser conectadas usando "&" e "|", e uma expressão pode ser negada usando "~".

In [39]:
despachantes.select("id", "nome", "vendas")\
        .where((f.col("vendas") > 20) & (f.col("vendas") < 40))\
        .show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



### Renomeando colunas:
> Diferente do pandas, onde podemos renomear diversas colunas passando um dicionário como parâmetro para o método da classe DataFrame; as colunas do DataFrame do PySpark devem ser renomeadas uma a uma. Portanto, faz-se necessário o uso de um loop, caso queiramos renomear mais de uma de forma ágil, por exemplo.

In [85]:
#renomeando todas as colunas de uma só vez numa nova variável
# novo_desp
novo_desp = desp_autoschema
for i in list(zip(desp_autoschema.columns, despachantes.columns)):
  novo_desp = novo_desp.withColumnRenamed(*i)

novo_desp.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



### Criando uma nova coluna com os dados da coluna data com o tipo timestamp

In [57]:
novo_desp = novo_desp.withColumn("data2", f.to_timestamp(f.col("data"), 
                                "yyyy-MM-dd"))
novo_desp.show()

+---+-------------------+------+-------------+------+----------+-------------------+
| id|               nome|status|       cidade|vendas|      data|              data2|
+---+-------------------+------+-------------+------+----------+-------------------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|2020-08-11 00:00:00|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|2020-03-05 00:00:00|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|2020-02-05 00:00:00|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|2020-02-05 00:00:00|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|2020-02-05 00:00:00|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|2019-01-05 00:00:00|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|2019-10-05 00:00:00|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|2020-03-05 00:00:00|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|2

In [58]:
novo_desp.schema

StructType(List(StructField(id,IntegerType,true),StructField(nome,StringType,true),StructField(status,StringType,true),StructField(cidade,StringType,true),StructField(vendas,IntegerType,true),StructField(data,StringType,true),StructField(data2,TimestampType,true)))

> Selecionando os anos das datas (str e timestamp) e os nomes dos despachantes, ordenando por nome.

In [59]:
novo_desp.select(f.year("data"), f.year("data2"), "nome")\
                .distinct()\
                .orderBy("nome")\
                .show()

+----------+-----------+-------------------+
|year(data)|year(data2)|               nome|
+----------+-----------+-------------------+
|      2020|       2020|   Carminda Pestana|
|      2020|       2020|    Deolinda Vilela|
|      2020|       2020|   Emídio Dornelles|
|      2020|       2020|Felisbela Dornelles|
|      2020|       2020|     Graça Ornellas|
|      2019|       2019|   Matilde Rebouças|
|      2019|       2019|    Noêmia   Orriça|
|      2020|       2020|      Roque Vásquez|
|      2018|       2018|      Uriel Queiroz|
|      2020|       2020|   Viviana Sequeira|
+----------+-----------+-------------------+



> Usando o ALIAS, podemos dar apelidos para todas as colunas, logo, podemos criar um agrupamento usando as funções do pacote functions e as apelidando e usando a referência do apelido em outros métodos do DataFrame como o groupBy e o orderBy.

In [79]:
novo_desp.select(f.year("data").alias("anos"))\
                .groupBy("anos")\
                .agg(f.count(f.col("anos")).alias("ocorr"))\
                .orderBy(f.col("ocorr").desc())\
                .show()#Só funciona se dermos um apelido para a coluna

+----+-----+
|anos|ocorr|
+----+-----+
|2020|    7|
|2019|    2|
|2018|    1|
+----+-----+



> Total de vendas

In [81]:
novo_desp.select(f.sum("vendas")).show()

+-----------+
|sum(vendas)|
+-----------+
|        325|
+-----------+

